# Data Aggregation and Group Operations

## GroupBy Mechanics

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.449500,0.177804
1,a,two,-1.299757,1.121115
2,b,one,-1.114697,-0.442910
3,b,two,-0.091343,-0.057761
4,a,one,0.966020,-1.936912


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a    0.038588
b   -0.603020
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     one     0.707760
      two    -1.299757
b     one    -1.114697
      two    -0.091343
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.707760,-1.299757
b,-1.114697,-0.091343


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [8]:
df["data1"].groupby([states, years]).mean()

California  2005   -1.299757
            2006   -1.114697
Ohio        2005    0.179079
            2006    0.966020
Name: data1, dtype: float64

If the grouping information is found in the same DataFrame as the data we want to work with. Column names are passed as the group keys.

In [9]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,0.038588,-0.212664
b,-0.603020,-0.250336


In [10]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one   0.707760 -0.879554
     two  -1.299757  1.121115
b    one  -1.114697 -0.442910
     two  -0.091343 -0.057761

In [11]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [12]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.449500  0.177804
1    a  two -1.299757  1.121115
4    a  one  0.966020 -1.936912
b
  key1 key2     data1     data2
2    b  one -1.114697 -0.442910
3    b  two -0.091343 -0.057761


In [13]:
for (k1,k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2    data1     data2
0    a  one  0.44950  0.177804
4    a  one  0.96602 -1.936912
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.299757  1.121115
('b', 'one')
  key1 key2     data1    data2
2    b  one -1.114697 -0.44291
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.091343 -0.057761


By default **groupby** groups on axis=0, but it can be done on any of the other axes.

In [14]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)

In [16]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.449500  0.177804
1 -1.299757  1.121115
2 -1.114697 -0.442910
3 -0.091343 -0.057761
4  0.966020 -1.936912
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [17]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    one  -0.879554
     two   1.121115
b    one  -0.442910
     two  -0.057761

In [18]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [19]:
s_grouped.mean()

key1  key2
a     one    -0.879554
      two     1.121115
b     one    -0.442910
      two    -0.057761
Name: data2, dtype: float64

### Grouping with Dict and Series

In [20]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.316905,-0.472760,0.721993,-1.533901,0.212607
Steve,-1.224632,-1.659532,0.114297,0.077546,0.383864
Wes,0.272166,1.042298,-0.822403,1.188329,-1.039213
Jim,0.550709,-1.613895,0.083900,-0.265626,0.498253
Travis,1.379595,-1.537418,0.777962,0.409617,0.916450


In [21]:
# Add a few NA values
people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.316905,-0.472760,0.721993,-1.533901,0.212607
Steve,-1.224632,-1.659532,0.114297,0.077546,0.383864
Wes,0.272166,NaN,NaN,1.188329,-1.039213
Jim,0.550709,-1.613895,0.083900,-0.265626,0.498253
Travis,1.379595,-1.537418,0.777962,0.409617,0.916450


In [22]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [23]:
by_column = people.groupby(mapping, axis=1)
print(by_column)

In [24]:
by_column.sum()

,blue,red
Joe,-0.811907,0.056752
Steve,0.191843,-2.500300
Wes,1.188329,-0.767047
Jim,-0.181726,-0.564932
Travis,1.187580,0.758627


In [25]:
map_series = pd.Series(mapping)

In [26]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [27]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [28]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.139780,-2.086655,0.805894,-0.611198,-0.328352
5,-1.224632,-1.659532,0.114297,0.077546,0.383864
6,1.379595,-1.537418,0.777962,0.409617,0.916450


In [29]:
key_list = ["one", 'one', "one", "two", "two"]

In [30]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.272166 -0.472760  0.721993 -1.533901 -1.039213
  two  0.550709 -1.613895  0.083900 -0.265626  0.498253
5 one -1.224632 -1.659532  0.114297  0.077546  0.383864
6 two  1.379595 -1.537418  0.777962  0.409617  0.916450

### Grouping by Index Levels

In [31]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [32]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)

In [33]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.533581  0.226755  1.155261 -0.386613 -0.284466
1      2.670679  1.120357  0.575727 -3.005444 -1.770391
2      0.062666 -1.900597 -0.244548 -0.535171  1.453230
3      0.970988 -0.130578 -0.202904  0.340525  0.485249

## Data Aggregation

In [34]:
df

,key1,key2,data1,data2
0,a,one,0.449500,0.177804
1,a,two,-1.299757,1.121115
2,b,one,-1.114697,-0.442910
3,b,two,-0.091343,-0.057761
4,a,one,0.966020,-1.936912


In [35]:
grouped = df.groupby("key1")
grouped

In [36]:
grouped["data1"].quantile(0.9)

key1
a    0.862716
b   -0.193678
Name: data1, dtype: float64

To use own aggregation functions, pass any function that aggregates an array to the **aggregate** or **agg** method.

In [37]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.265778,3.058027
b,1.023355,0.385148


In [38]:
grouped.describe()

data1                                                             \
     count      mean       std       min       25%      50%       75%   
key1                                                                    
a      3.0  0.038588  1.187466 -1.299757 -0.425129  0.44950  0.707760   
b      2.0 -0.603020  0.723621 -1.114697 -0.858859 -0.60302 -0.347181   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.966020   3.0 -0.212664  1.565960 -1.936912 -0.879554  0.177804   
b    -0.091343   2.0 -0.250336  0.272341 -0.442910 -0.346623 -0.250336   

                          
           75%       max  
key1                      
a     0.649459  1.121115  
b    -0.154048 -0.057761

### Column-Wise and Multiple Function Application

In [39]:
tips = pd.read_csv("../examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [40]:
# Add tip percentage of total bill

tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [41]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [42]:
grouped = tips.groupby(["day", "smoker"])
grouped

In [43]:
grouped_pct = grouped["tip_pct"]

In [44]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [45]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [46]:
grouped_pct.agg([("foo","mean"), ("bar",np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame there are more options, as the list of functions can be specified per column.

In [47]:
functions = ["count", "mean", "max"]

In [48]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/Library/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

If you want to apply potentially different functions to one or more of the columns. Pass a **dict** to **agg** that contains a mapping of column names to any of the function specifications listed so far.

In [49]:
grouped.agg({"tip":np.max, "size":"sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [51]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [52]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [53]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [54]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [55]:
result = tips.groupby("smoker")["tip_pct"].describe()

In [56]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [57]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Supressing the Group Keys

In [58]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

Pandas has some tools, in particular **cut** and **qcut**, for slicing data up into buckets with bins of your choosing or by sample quantiles.

In [59]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame

,data1,data2
0,0.466137,-1.325524
1,0.836199,0.295398
2,-1.061347,-0.944702
3,0.918760,-0.471489
4,-0.214150,1.640769
...,...,...
995,-0.036368,1.082372
996,-0.322585,-1.124387
997,-0.380016,-0.319043
998,0.333835,-0.728717


In [61]:
quartiles = pd.cut(frame.data1, 4)
quartiles

0      (-1.045, 0.494]
1       (0.494, 2.034]
2      (-2.59, -1.045]
3       (0.494, 2.034]
4      (-1.045, 0.494]
            ...       
995    (-1.045, 0.494]
996    (-1.045, 0.494]
997    (-1.045, 0.494]
998    (-1.045, 0.494]
999    (-1.045, 0.494]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.59, -1.045] < (-1.045, 0.494] < (0.494, 2.034] < (2.034, 3.573]]

In [64]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.59, -1.045]",-2.852822,1.958960,148.0,-0.073446
"(-1.045, 0.494]",-3.751301,2.879016,519.0,0.018451
"(0.494, 2.034]",-2.967192,3.498015,307.0,-0.052912
"(2.034, 3.573]",-1.387509,1.631028,26.0,-0.040725


To compute equal-size buckets based on sample quantiles, use **qcut**. Pass labels=Fa;se to just get quantile numbers.

In [65]:
# Return quantile numbers

grouping = pd.qcut(frame.data1, 10, labels=False)

In [66]:
grouped = frame.data2.groupby(grouping)

In [67]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.852822,1.957284,100.0,-0.141465
1,-2.885215,2.752636,100.0,0.031651
2,-3.751301,2.227727,100.0,0.005511
3,-2.870231,2.486253,100.0,0.047528
4,-2.487896,2.879016,100.0,-0.043046
5,-2.397155,2.242151,100.0,0.073648
6,-2.267720,2.718169,100.0,0.049810
7,-2.967192,3.498015,100.0,-0.087958
8,-2.706599,2.872490,100.0,-0.037905


## Pivot Tables and Cross-Tabulation

A **pivot table** is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.  In addition to providing a convenience interface to groupby, pivot_table can add partial totals, also known as margins.
 

In [68]:
tips.pivot_table(index=["day", "smoker"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [70]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"], columns="smoker")

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

### Cross-Tabulations: Crosstab

A cross-tabulation is a special case of a pivot table that computes group frequencies.

In [72]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [73]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [74]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [75]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244